In [1]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
#pcd = o3d.io.read_point_cloud("kitchen.ply")
pcd = o3d.io.read_point_cloud("filtered.pcd")

In [3]:
pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=16), fast_normal_computation=True)
pcd.paint_uniform_color([0.6, 0.6, 0.6])

PointCloud with 172285 points.

In [4]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(inliers, invert=True)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud.paint_uniform_color([0.6, 0.6, 0.6])

Plane equation: 0.23x + 0.70y + 0.68z + -0.84 = 0


PointCloud with 94094 points.

In [9]:
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

In [5]:
labels = np.array(pcd.cluster_dbscan(eps=0.05, min_points=4))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab20")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
pcd.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 1 clusters


In [6]:
segment_models={}
segments={}
max_plane_idx=4
rest=pcd
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    segments[i].paint_uniform_color(list(colors[:3]))
    rest = rest.select_by_index(inliers, invert=True)
    print("pass",i,"/",max_plane_idx,"done.")

pass 0 / 4 done.
pass 1 / 4 done.
pass 2 / 4 done.
pass 3 / 4 done.


In [7]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest])

In [16]:
segment_models={}
segments={}
max_plane_idx=20
rest=pcd
d_threshold=0.01
for i in range(max_plane_idx):
    colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=0.01,ransac_n=3,num_iterations=1000)
    segments[i]=rest.select_by_index(inliers)
    labels = np.array(segments[i].cluster_dbscan(eps=d_threshold*10, min_points=10))
    candidates=[len(np.where(labels==j)[0]) for j in np.unique(labels)]
    best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])
    print("the best candidate is: ", best_candidate)
    rest = rest.select_by_index(inliers, invert=True)+segments[i].select_by_index(list(np.where(labels!=best_candidate)[0]))
    segments[i]=segments[i].select_by_index(list(np.where(labels==best_candidate)[0]))
    segments[i].paint_uniform_color(list(colors[:3]))
    print("pass",i+1,"/",max_plane_idx,"done.")

/tmp/ipykernel_23790/1834253068.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate=int(np.unique(labels)[np.where(candidates==np.max(candidates))[0]])


the best candidate is:  0
pass 1 / 20 done.
the best candidate is:  0
pass 2 / 20 done.
the best candidate is:  0
pass 3 / 20 done.
the best candidate is:  0
pass 4 / 20 done.
the best candidate is:  0
pass 5 / 20 done.
the best candidate is:  0
pass 6 / 20 done.
the best candidate is:  0
pass 7 / 20 done.
the best candidate is:  0
pass 8 / 20 done.
the best candidate is:  0
pass 9 / 20 done.
the best candidate is:  0
pass 10 / 20 done.
the best candidate is:  -1
pass 11 / 20 done.


RuntimeError: [1;31m[Open3D Error] (std::tuple<Eigen::Matrix<double, 4, 1, 0, 4, 1>, std::vector<long unsigned int, std::allocator<long unsigned int> > > open3d::geometry::PointCloud::SegmentPlane(double, int, int, double) const) /root/Open3D/cpp/open3d/geometry/PointCloudSegmentation.cpp:175: There must be at least 'ransac_n' points.
[0;m

In [26]:
labels = np.array(rest.cluster_dbscan(eps=0.05, min_points=5))
max_label = labels.max()
print(f"point cloud has {max_label + 1} clusters")

colors = plt.get_cmap("tab10")(labels / (max_label if max_label > 0 else 1))
colors[labels < 0] = 0
rest.colors = o3d.utility.Vector3dVector(colors[:, :3])

point cloud has 98 clusters


In [27]:
o3d.visualization.draw_geometries([segments[i] for i in range(max_plane_idx)]+[rest],zoom=0.3199,front=[0.30159062875123849, 0.94077325609922868, 0.15488309545553303],lookat=[-3.9559999108314514, -0.055000066757202148, -0.27599999308586121],up=[-0.044411423633999815, -0.138726419067636, 0.98753122516983349])